# 실습 06: 이미지 전처리로 OCR 정확도 향상

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab06_이미지전처리.ipynb)

## 🎯 학습 목표
- 이미지 품질 평가
- 노이즈 제거, 이진화, 기울기 보정
- 전처리 전후 OCR 성능 비교

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 노이즈가 있는 테스트 이미지 생성
from docai_course.preprocessing import DocumentPreprocessor
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

# 한글 폰트 로드
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

# 깨끗한 이미지 생성
img = Image.new('RGB', (600, 300), color='white')
draw = ImageDraw.Draw(img)
text = "계약서\n\n계약일자: 2025년 1월 15일\n계약금액: 1,500,000원\n담당자: 홍길동 (02-1234-5678)"
draw.text((30, 30), text, fill='black', font=font)

# OpenCV 변환 및 노이즈 추가 (더 강하게)
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 40, img_cv.shape).astype(np.uint8)
noisy_img = cv2.add(img_cv, noise)

# 기울기 추가 (7도 - 더 명확하게)
rows, cols = noisy_img.shape[:2]
M = cv2.getRotationMatrix2D((cols/2, rows/2), 7, 1)
rotated_img = cv2.warpAffine(noisy_img, M, (cols, rows), borderValue=(255,255,255))

# 추가: 약간의 블러 효과
blurred_img = cv2.GaussianBlur(rotated_img, (3, 3), 0)
cv2.imwrite('noisy_document.jpg', blurred_img)

# 원본도 저장 (비교용)
cv2.imwrite('clean_document.jpg', img_cv)

print("✅ 노이즈 이미지 생성 완료!")


## 💡 전처리 파이프라인 개요

이미지 전처리는 **단계별로 진행**됩니다:

### 1️⃣ 노이즈 제거 (Denoising)
- **목적**: 스캔 과정에서 생긴 노이즈 제거
- **방법**: fastNlMeansDenoising (Non-Local Means)
- **효과**: 깨끗한 이미지로 OCR 정확도 향상

### 2️⃣ 기울기 보정 (Deskewing)
- **목적**: 기울어진 문서를 수평으로 정렬
- **방법**: 최소 영역 사각형(Minimum Area Rectangle)으로 각도 계산
- **효과**: 텍스트 인식 정확도 향상

### 3️⃣ 이진화 (Binarization)
- **목적**: 텍스트와 배경을 명확히 구분
- **방법**: 적응형 임계값(Adaptive Threshold)
- **효과**: OCR 엔진이 텍스트 경계를 명확히 인식

**각 단계를 순차적으로 적용하며 효과를 측정합니다!**


In [ ]:
# 단계 2: 기울기 보정 (Deskewing)
print("\n" + "=" * 70)
print("🔧 단계 2: 기울기 보정")
print("=" * 70)

# ⚠️ 개선: THRESH_BINARY_INV로 텍스트를 흰색으로 만들기
_, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 텍스트 픽셀 좌표 추출 (이제 흰색=텍스트)
coords = np.column_stack(np.where(binary > 0))

if len(coords) == 0:
    print("  ⚠️ 텍스트 픽셀을 찾을 수 없습니다. 기울기 보정 생략.")
    deskewed = denoised.copy()
else:
    # 최소 영역 사각형으로 각도 계산
    rect = cv2.minAreaRect(coords)
    angle = rect[-1]
    
    # 각도 정규화 (-45 ~ +45 범위로 변환)
    if angle < -45:
        angle = 90 + angle
    
    print(f"\n📐 기울기 감지:")
    print(f"  minAreaRect 반환 각도: {rect[-1]:.2f}도")
    print(f"  정규화된 각도: {angle:.2f}도")
    print(f"  → 보정 회전 각도: {-angle:.2f}도 (반대 방향)")
    
    # 기울기 보정 적용 (반대 방향으로 회전)
    if abs(angle) > 0.5:  # 0.5도 이상만 보정
        (h, w) = denoised.shape
        center = (w // 2, h // 2)
        # ⚠️ 핵심 수정: -angle로 반대 방향 회전해야 바로잡힘
        M = cv2.getRotationMatrix2D(center, -angle, 1.0)
        deskewed = cv2.warpAffine(denoised, M, (w, h), 
                                  flags=cv2.INTER_CUBIC, 
                                  borderMode=cv2.BORDER_REPLICATE)
        print(f"  ✅ 보정 적용: {-angle:.2f}도 회전")
    else:
        deskewed = denoised.copy()
        print(f"  ✅ 보정 불필요 (각도가 미세함)")

cv2.imwrite('step2_deskewed.jpg', deskewed)
print("\n✅ 기울기 보정 완료!")


In [ ]:
# 전처리 단계별 시각화
import matplotlib.pyplot as plt

# 모든 단계 이미지 로드
original = cv2.imread('noisy_document.jpg', cv2.IMREAD_GRAYSCALE)
step1 = cv2.imread('step1_denoised.jpg', cv2.IMREAD_GRAYSCALE)
step2 = cv2.imread('step2_deskewed.jpg', cv2.IMREAD_GRAYSCALE)
step3 = cv2.imread('step3_binary.jpg', cv2.IMREAD_GRAYSCALE)

# 2x2 그리드로 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(original, cmap='gray')
axes[0, 0].set_title('① 원본 (노이즈 + 기울기)', fontsize=14, fontproperties=fontprop, pad=15)
axes[0, 0].axis('off')

axes[0, 1].imshow(step1, cmap='gray')
axes[0, 1].set_title('② 노이즈 제거 후', fontsize=14, fontproperties=fontprop, pad=15)
axes[0, 1].axis('off')

axes[1, 0].imshow(step2, cmap='gray')
axes[1, 0].set_title('③ 기울기 보정 후', fontsize=14, fontproperties=fontprop, pad=15)
axes[1, 0].axis('off')

axes[1, 1].imshow(step3, cmap='gray')
axes[1, 1].set_title('④ 이진화 후 (최종)', fontsize=14, fontproperties=fontprop, pad=15)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ 전처리 단계별 시각화 완료!")


In [ ]:
# 단계 2: 기울기 보정 (Deskewing)
print("\n" + "=" * 70)
print("🔧 단계 2: 기울기 보정")
print("=" * 70)

# 이진화 (기울기 감지용)
_, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 텍스트 픽셀 좌표 추출
coords = np.column_stack(np.where(binary < 255))

# 최소 영역 사각형으로 각도 계산
rect = cv2.minAreaRect(coords)
angle = rect[-1]

# 각도 정규화 (-45 ~ +45 범위)
if angle < -45:
    angle = 90 + angle

print(f"\n📐 기울기 감지:")
print(f"  감지된 각도: {angle:.2f}도")

# 기울기 보정 적용
if abs(angle) > 0.5:  # 0.5도 이상만 보정
    (h, w) = denoised.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskewed = cv2.warpAffine(denoised, M, (w, h), 
                              flags=cv2.INTER_CUBIC, 
                              borderMode=cv2.BORDER_REPLICATE)
    print(f"  보정 적용: {angle:.2f}도 회전")
else:
    deskewed = denoised.copy()
    print(f"  보정 불필요 (각도가 미세함)")

cv2.imwrite('step2_deskewed.jpg', deskewed)
print("\n✅ 기울기 보정 완료!")


In [ ]:
# 단계 3: 적응형 이진화 (Adaptive Binarization)
print("\n" + "=" * 70)
print("🔧 단계 3: 적응형 이진화")
print("=" * 70)

# 일반 이진화 (비교용)
_, binary_simple = cv2.threshold(deskewed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 적응형 이진화 (추천)
binary_adaptive = cv2.adaptiveThreshold(
    deskewed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    cv2.THRESH_BINARY, 11, 2
)

print(f"\n📊 이진화 방법 비교:")
print(f"  • 일반 이진화: 전체 이미지에 단일 임계값 적용")
print(f"  • 적응형 이진화: 지역별로 다른 임계값 적용 (조명 불균형에 강함)")

cv2.imwrite('step3_binary.jpg', binary_adaptive)
print("\n✅ 이진화 완료!")


In [ ]:
# 단계별 OCR 정확도 비교
import pytesseract
import Levenshtein

ground_truth = "계약서 계약일자: 2025년 1월 15일 계약금액: 1,500,000원 담당자: 홍길동 (02-1234-5678)"

print("\n" + "="*70)
print("📊 단계별 OCR 정확도 측정")
print("="*70)

# 정확도 계산 함수
def calc_accuracy(pred, truth):
    pred_clean = pred.replace(' ', '').replace('\n', '')
    truth_clean = truth.replace(' ', '').replace('\n', '')
    distance = Levenshtein.distance(pred_clean, truth_clean)
    return max(0, (1 - distance / max(len(pred_clean), len(truth_clean))) * 100)

# 각 단계별 OCR 실행
results = []

# 원본 (노이즈 이미지)
img_original = cv2.imread('noisy_document.jpg')
text_original = pytesseract.image_to_string(img_original, lang='kor+eng').strip()
acc_original = calc_accuracy(text_original, ground_truth)
results.append(('원본 (노이즈)', acc_original, text_original))

# 단계 1: 노이즈 제거
img_denoised = cv2.imread('step1_denoised.jpg')
text_denoised = pytesseract.image_to_string(img_denoised, lang='kor+eng').strip()
acc_denoised = calc_accuracy(text_denoised, ground_truth)
results.append(('① 노이즈 제거', acc_denoised, text_denoised))

# 단계 2: 기울기 보정
img_deskewed = cv2.imread('step2_deskewed.jpg')
text_deskewed = pytesseract.image_to_string(img_deskewed, lang='kor+eng').strip()
acc_deskewed = calc_accuracy(text_deskewed, ground_truth)
results.append(('② 기울기 보정', acc_deskewed, text_deskewed))

# 단계 3: 이진화
img_binary = cv2.imread('step3_binary.jpg')
text_binary = pytesseract.image_to_string(img_binary, lang='kor+eng').strip()
acc_binary = calc_accuracy(text_binary, ground_truth)
results.append(('③ 이진화', acc_binary, text_binary))

# 결과 출력
print("\n단계별 OCR 결과:")
for step_name, acc, text in results:
    print(f"\n{step_name}:")
    print(f"  정확도: {acc:.2f}%")
    print(f"  인식 텍스트: {text[:50]}...")

# 개선 효과 계산
total_improvement = acc_binary - acc_original
print("\n" + "="*70)
print(f"🎉 전체 개선 효과: {acc_original:.1f}% → {acc_binary:.1f}% (+{total_improvement:.1f}%p)")
print("="*70)

# 시각화
fig, ax = plt.subplots(figsize=(12, 6))
stages = [r[0] for r in results]
accuracies = [r[1] for r in results]
colors = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']

bars = ax.bar(stages, accuracies, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('정확도 (%)', fontproperties=fontprop, fontsize=12)
ax.set_title('전처리 단계별 OCR 정확도 향상', fontsize=16, fontproperties=fontprop, pad=20)
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)
ax.axhline(y=acc_original, color='red', linestyle='--', alpha=0.5, label=f'원본: {acc_original:.1f}%')

# 막대 위에 수치 표시
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    improvement = acc - acc_original
    label_text = f'{acc:.1f}%\n(+{improvement:.1f}%p)' if improvement != 0 else f'{acc:.1f}%'
    ax.text(bar.get_x() + bar.get_width()/2., height + 2,
            label_text,
            ha='center', va='bottom', fontsize=11, fontweight='bold', fontproperties=fontprop)

plt.legend(prop=fontprop)
plt.tight_layout()
plt.show()

# 각 전처리 단계의 효과 분석
print("\n💡 전처리 단계별 효과 분석:")
for i in range(1, len(results)):
    prev_acc = results[i-1][1]
    curr_acc = results[i][1]
    diff = curr_acc - prev_acc
    print(f"  {results[i][0]}: {'+' if diff > 0 else ''}{diff:.1f}%p")

print("\n✅ 단계별 전처리 효과 분석 완료!")
